<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/kc_11aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 7.4 MB/s eta 0:00:00


In [2]:
from kiteconnect import KiteConnect
import pandas as pd
import logging
import os
import datetime as dt
api_key = "1ll4p8ziqlk8hh7g"
api_secret = "x59r5u1vd25annfpgts22wv8ykfueirl"
kite = KiteConnect(api_key=api_key)

In [3]:
print(kite.login_url()) #use this url to manually login and authorize yourself

https://kite.zerodha.com/connect/login?api_key=1ll4p8ziqlk8hh7g&v=3


In [5]:
#generate trading session
request_token = "ns3PGi8Wb8gl2Rn4qICas4b2bJb1XNB9" #Extract request token from the redirect url obtained after you authorize yourself by loggin in
data = kite.generate_session(request_token, api_secret=api_secret)
#create kite trading object
kite.set_access_token(data["access_token"])

In [6]:
#get dump of all NSE instruments
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv("NSE_Instruments.csv",index=False)

In [7]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [8]:
def fetchOHLC(ticker,interval,duration):
  """extracts historcial data and outputs in the form of dataframe"""
  instrument = instrumentLookup(instrument_df,ticker)
  data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration),dt.date.today(),interval))
  data.set_index("date",inplace=True)
  return data

# **MACD**

In [9]:
def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a(fast moving average) = 12;
                      b(slow moving average) =26;
                      c(signal line ma window) =9"""
    df = DF.copy()
    df["MA_Fast"]=df["close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df

In [10]:
ohlc  = fetchOHLC("INFY", "5minute", 2)
macd  = MACD(ohlc,12,26,9)
macd

,open,high,low,close,volume,MA_Fast,MA_Slow,MACD,Signal
date,,,,,,,,,
2025-08-11 12:00:00+05:30,1424.3,1424.9,1423.0,1423.6,41839,1424.196378,1423.149473,1.046905,1.351532
2025-08-11 12:05:00+05:30,1423.5,1425.0,1422.8,1424.8,40241,1424.289512,1423.280603,1.008909,1.274764
2025-08-11 12:10:00+05:30,1424.8,1426.2,1424.5,1425.5,42699,1424.476197,1423.455986,1.020211,1.219069
2025-08-11 12:15:00+05:30,1425.4,1425.8,1424.9,1425.0,23180,1424.556949,1423.577398,0.979551,1.167631
2025-08-11 12:20:00+05:30,1425.0,1425.9,1424.9,1425.2,23719,1424.656053,1423.704410,0.951644,1.121921
...,...,...,...,...,...,...,...,...,...
2025-08-13 15:05:00+05:30,1428.0,1428.1,1426.2,1427.3,363485,1428.917735,1428.584995,0.332740,0.685173
2025-08-13 15:10:00+05:30,1427.2,1427.5,1425.8,1425.9,327441,1428.453468,1428.386107,0.067361,0.561611
2025-08-13 15:15:00+05:30,1425.8,1426.9,1425.5,1426.1,295526,1428.091396,1428.216766,-0.125370,0.424215


# **Bollinger Band**

In [11]:
def bollBnd(DF,n):
    "function to calculate Bollinger Band"
    df = DF.copy()
    df["MA"] = df['close'].rolling(n).mean()
    #df["MA"] = df['close'].ewm(span=n,min_periods=n).mean()
    df["BB_up"] = df["MA"] + 2*df['close'].rolling(n).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    df["BB_dn"] = df["MA"] - 2*df['close'].rolling(n).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    df["BB_width"] = df["BB_up"] - df["BB_dn"]
    df.dropna(inplace=True)
    return df

In [12]:
bb = bollBnd(ohlc,20)
bb

,open,high,low,close,volume,MA,BB_up,BB_dn,BB_width
date,,,,,,,,,
2025-08-11 10:50:00+05:30,1422.1,1424.5,1421.8,1424.2,77329,1418.960,1424.715484,1413.204516,11.510969
2025-08-11 10:55:00+05:30,1424.2,1424.5,1421.5,1421.6,55991,1418.850,1424.307655,1413.392345,10.915310
2025-08-11 11:00:00+05:30,1421.6,1422.1,1420.8,1422.1,23450,1418.745,1423.856741,1413.633259,10.223483
2025-08-11 11:05:00+05:30,1422.1,1425.4,1422.0,1423.9,64157,1418.950,1424.522612,1413.377388,11.145223
2025-08-11 11:10:00+05:30,1424.0,1424.1,1422.0,1423.3,42443,1419.215,1425.078199,1413.351801,11.726398
...,...,...,...,...,...,...,...,...,...
2025-08-13 15:05:00+05:30,1428.0,1428.1,1426.2,1427.3,363485,1428.930,1431.602153,1426.257847,5.344305
2025-08-13 15:10:00+05:30,1427.2,1427.5,1425.8,1425.9,327441,1428.835,1431.782049,1425.887951,5.894099
2025-08-13 15:15:00+05:30,1425.8,1426.9,1425.5,1426.1,295526,1428.740,1431.903163,1425.576837,6.326326


# **ATR**

In [13]:
def atr(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['high']-df['low'])
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

In [14]:
attr = atr(ohlc,20)
attr

,ATR
date,
2025-08-11 09:15:00+05:30,NaN
2025-08-11 09:20:00+05:30,NaN
2025-08-11 09:25:00+05:30,NaN
2025-08-11 09:30:00+05:30,NaN
2025-08-11 09:35:00+05:30,NaN
...,...
2025-08-13 15:05:00+05:30,1.885401
2025-08-13 15:10:00+05:30,1.876572
2025-08-13 15:15:00+05:30,1.853878


# **RSI**

In [15]:
def rsi(df, n):
    "function to calculate RSI"
    delta = df["close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

In [16]:
import numpy as np
rssi = rsi(ohlc,14)
rssi

,close
date,
2025-08-11 10:25:00+05:30,NaN
2025-08-11 10:30:00+05:30,NaN
2025-08-11 10:35:00+05:30,NaN
2025-08-11 10:40:00+05:30,NaN
2025-08-11 10:45:00+05:30,NaN
...,...
2025-08-13 15:05:00+05:30,44.401469
2025-08-13 15:10:00+05:30,39.211680
2025-08-13 15:15:00+05:30,40.280087


# **ADX**

In [17]:
def adx(DF,n):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['H-L']=abs(df2['high']-df2['low'])
    df2['H-PC']=abs(df2['high']-df2['close'].shift(1))
    df2['L-PC']=abs(df2['low']-df2['close'].shift(1))
    df2['TR']=df2[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df2['DMplus']=np.where((df2['high']-df2['high'].shift(1))>(df2['low'].shift(1)-df2['low']),df2['high']-df2['high'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['low'].shift(1)-df2['low'])>(df2['high']-df2['high'].shift(1)),df2['low'].shift(1)-df2['low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']

# **Mutual Fund**

In [18]:
my_mf  = pd.DataFrame(kite.mf_holdings())
my_mf

,tradingsymbol,fund,folio,last_price,last_price_date,quantity,pnl,xirr,average_price,discrepancy,pledged_quantity
0,INF879O01100,PARAG PARIKH ELSS TAX SAVER FUND - DIRECT PLAN,10667243,33.4938,,1125.897,0.0,0.0,18.650861,False,0.0
1,INF966L01986,QUANT ELSS TAX SAVER FUND - DIRECT PLAN,5102777842,392.6650,,32.351,0.0,0.0,278.174291,False,0.0
2,INF200KA1S55,SBI RETIREMENT BENEFIT FUND - AGGRESSIVE PLAN ...,25466449,20.5655,,3980.011,0.0,0.0,13.567118,False,0.0
3,INF740KA1TW1,DSP NIFTY SMALLCAP250 QUALITY 50 INDEX FUND - ...,6357485/73,11.6244,,721.043,0.0,0.0,11.233293,False,0.0
4,INF760K01EL8,CANARA ROBECO ELSS TAX SAVER FUND - DIRECT PLAN,18836508181,191.4700,,230.337,0.0,0.0,117.213627,False,0.0


In [ ]:
all_mf = pd.DataFrame(kite.mf_instruments())
all_mf.head(5)

,tradingsymbol,amc,name,purchase_allowed,redemption_allowed,minimum_purchase_amount,purchase_amount_multiplier,minimum_additional_purchase_amount,minimum_redemption_quantity,redemption_quantity_multiplier,dividend_type,scheme_type,plan,settlement_type,last_price,last_price_date
0,INF00XX01135,ITI MUTUAL FUND_MF,ITI Multi Cap Fund,False,True,1000.0,1.0,1000.0,0.001,0.001,growth,Equity,regular,T2,22.7888,2025-08-08
1,INF00XX01143,ITI MUTUAL FUND_MF,ITI Multi Cap Fund,False,True,1000.0,1.0,1000.0,0.001,0.001,idcw-payout,Equity,regular,T2,20.8907,2025-08-08
2,INF00XX01168,ITI MUTUAL FUND_MF,ITI Multi Cap Fund - Direct Plan,True,True,1000.0,1.0,1000.0,0.001,0.001,growth,Equity,direct,T2,25.7734,2025-08-08
3,INF00XX01176,ITI MUTUAL FUND_MF,ITI Multi Cap Fund - Direct Plan,True,True,1000.0,1.0,1000.0,0.001,0.001,idcw-payout,Equity,direct,T2,23.7510,2025-08-08
4,INF00XX01192,ITI MUTUAL FUND_MF,ITI Liquid Fund,False,True,5000.0,1.0,1000.0,0.001,0.001,growth,Debt,regular,T1,1369.1828,2025-08-10
